In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pygam import LinearGAM, s, f
from sklearn.model_selection import GridSearchCV

In [6]:
df = pd.read_csv('assets/output/preprocessedDataset.csv')

In [7]:
# df_temp = df.drop('date', axis=1)
# df_temp['visitor'] = df_temp['visitor'].str.replace(',', '')
# df_temp['visitor'] = df_temp['visitor'].astype(int)

In [8]:
X = df.drop(['date', 'visitor'], axis=1)
y = df['visitor']

In [9]:
# # 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#
# # 모델 정의
# gam = LinearGAM()
#
# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'lam': np.arange(0.1, 1.1, 0.1),
#     'n_splines': np.arange(10, 31, 10),
#     'degree': np.arange(1, 4)
# }
#
# # 그리드 서치 객체 생성
# grid_search = GridSearchCV(gam, param_grid=param_grid, cv=5, return_train_score=True)
#
# # 그리드 서치 수행
# grid_search.fit(X_train, y_train)
#
# # 최적 매개변수 조합 및 최적 모델 출력
# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)
# best_model = grid_search.best_estimator_
# print("Best Model:", best_model)
#
# # 테스트 데이터에 대한 예측
# y_pred = best_model.predict(X_test)
#
# # 평가: 평균 제곱 오차 계산
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")

In [10]:
selected_feat = ['sulfur_dioxide_min', 'carbon_monoxide_max', 'ozone_max', 'nitrogen_dioxide_max', 'fine_dust_pm10_max',                 'rainfall_max', 'probability of precipitation_min', 'humidity_min', 'highest temperature_max',                 'lowest temperature_min', 'wind speed_median', 'visitor']
df_selected = df[selected_feat]

X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

In [16]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv('assets/output/preprocessedDataset.csv')

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 상수 열 추가
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# GAM 모델 생성 및 학습
gam = LinearGAM()
gam.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = gam.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 평가: 결정 계수(R^2) 계산
r2_gam = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2_gam}")

Mean Squared Error: 132526884.64813472
R^2 Score: 0.1301510131337621


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 로드
data = pd.read_csv('assets/output/preprocessedDataset.csv')

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

# 다항 특성 추가
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)

model.impo

print(f"Mean Squared Error: {mse}")

# # 평가: 결정 계수(R^2) 계산
# r2_lin = r2_score(X_test, y_test)
# print(f"R^2 Score: {r2_lin}")

score = model.score(X_test, y_test)
print(f"score: {score}")

Mean Squared Error: 145805482732.91013
score: -1090.3928561729356


In [13]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터를 k개의 부분집합으로 나누는 k-fold 객체 생성
k = 5  # k-fold 교차 검증에서의 k 값 설정
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# 모델 초기화
model = LinearRegression()

# k-fold 교차 검증 수행
mse_scores = []  # 각 fold에서의 평균 제곱 오차(MSE)를 저장할 리스트
score_arr = []

for train_index, val_index in kf.split(X_poly):
    # 훈련 세트, 검증 세트 나누기
    X_train, X_val = X_poly[train_index], X_poly[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트 예측
    y_val_pred = model.predict(X_val)

    # 검증 세트의 평균 제곱 오차 계산
    mse = mean_squared_error(y_val, y_val_pred)
    score = model.score(X_val, y_val)
    score_arr.append(score)
    mse_scores.append(mse)

# k-fold 교차 검증의 평균 MSE 계산
mean_mse = np.mean(mse_scores)
mean_score = np.mean(score_arr)

print(mean_score)

-0.4038758783528049
